In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [194]:
!pip install gensim==3.6.0
!pip install -U nltk
import numpy as np
import cv2
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
import re
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential , Model
from keras.layers import Dense, Embedding, GRU, LSTM ,SpatialDropout1D, Bidirectional, Dropout , Input 
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('popular')
nltk.download("stopwords")
import gensim
from gensim.models import Word2Vec 
import xlrd
stop_words=set(nltk.corpus.stopwords.words('arabic'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading 

In [32]:
test_dataset='/content/drive/MyDrive/NLP_3/all.dev.features.xlsx'
train_dataset ='/content/drive/MyDrive/NLP_3/all.test.features.xlsx'
x_train =[]
y_train =[]
x_test =[]
y_test =[]
train_data = xlrd.open_workbook(train_dataset)
train_data = train_data.sheet_by_index(0)
test_data = xlrd.open_workbook(test_dataset)
test_data = test_data.sheet_by_index(0)
for i in range(0,48287):
  x_train.append(train_data.cell_value(i, 0))
  y_train.append(train_data.cell_value(i, 36))
for i in range(0,20855):
  x_test.append(test_data.cell_value(i, 0))
  y_test.append(test_data.cell_value(i, 36))

In [4]:
print(y_train)
print(y_test)

['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', '', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'B-MIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O',

In [33]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(48287,)
(48287,)
(20855,)
(20855,)


In [7]:
arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [8]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

In [9]:
def remove_repeating_char(text):
    # return re.sub(r'(.)\1+', r'\1', text)     # keep only 1 repeat
    return re.sub(r'(.)\1+', r'\1\1', text)  # keep 2 repeat

In [10]:
def process_text(text,
                 remove_repeated_chars=False,
                 remove_diacritics=False,
                 ):
    clean_text = text
    clean_text =clean_text.lower()
    if remove_repeated_chars:
        clean_text = remove_repeating_char(clean_text)
    if remove_diacritics:
        clean_text = remove_diacritics(clean_text)
    return clean_text

In [34]:
for i in range(0,len(x_train)):
  x_train[i] = process_text(x_train[i])

In [35]:
unique_words=set(x_train)

In [13]:
print(len(unique_words))

13308


In [36]:
t  = Tokenizer()
t.fit_on_texts(x_train)
word_index = t.word_index
print("The word index", word_index)
sequences = t.texts_to_sequences(x_train)

The word index {'،': 1, 'في': 2, 'من': 3, '0': 4, 'و': 5, 'على': 6, 'إلى': 7, 'أن': 8, 'التي': 9, 'كان': 10, 'أو': 11, 'عام': 12, 'عن': 13, 'هذه': 14, 'القدم': 15, 'الذي': 16, 'مع': 17, 'ما': 18, 'ذلك': 19, 'كانت': 20, 'هذا': 21, 'العالم': 22, 'بعد': 23, 'هو': 24, 'بن': 25, 'بين': 26, 'لكرة': 27, 'حيث': 28, 'كل': 29, 'قد': 30, 'مثل': 31, 'لا': 32, 'الضوء': 33, 'لم': 34, 'وفي': 35, 'هي': 36, 'تم': 37, 'كما': 38, 'وقد': 39, 'كأس': 40, 'ثم': 41, 'البرتغال': 42, 'أي': 43, 'يمكن': 44, 'حتى': 45, 'أول': 46, 'منتخب': 47, 'غير': 48, 'القرن': 49, 'رونالدو': 50, 'بعض': 51, 'الجامع': 52, '1': 53, 'فيها': 54, 'وكان': 55, 'لكن': 56, 'سنة': 57, 'قبل': 58, 'أيضا': 59, 'كرة': 60, 'إن': 61, 'قام': 62, 'عدد': 63, 'عشر': 64, 'خلال': 65, 'أخرى': 66, 'أكثر': 67, 'فإن': 68, 'العناصر': 69, 'الرازي': 70, 'نظام': 71, 'وهي': 72, 'لها': 73, '3': 74, '2': 75, 'به': 76, 'بها': 77, 'البرمجيات': 78, 'ولكن': 79, 'آخر': 80, 'العديد': 81, 'الأولى': 82, 'عندما': 83, 'له': 84, 'س': 85, 'مجموعة': 86, 'يتم': 87, 'عليها': 8

In [37]:
vocab_size = len(word_index) + 1
print(len(word_index))

13262


In [112]:
print(sequences)
print(len(sequences))

[[1135], [50], [339], [2751], [4716], [1136], [2], [459], [47], [42], [27], [15], [5], [512], [6], [311], [339], [2], [22], [944, 4], [4717], [370], [4718], [3], [311], [3], [4719], [675], [2], [1137], [676], [27], [15], [1], [460], [183], [459], [184], [196], [513], [1420], [1], [41], [786], [580], [312], [1], [258], [2752], [313], [15], [83], [10], [2], [1421], [787], [55], [4720], [1891], [4721], [15], [139], [2753], [4722], [3], [2754], [], [], [258], [581], [17], [238], [2755], [1], [4723], [1], [83], [10], [788], [4724], [285], [123], [1], [55], [1138], [409], [4725], [4726], [2], [19], [677], [], [], [1139], [12], [2756, 4], [55], [30], [945], [4727], [3], [788], [1140], [4728], [4729], [2], [1141], [1892], [1893], [50], [7], [459], [4730], [677], [], [], [39], [4731], [340], [4732], [339], [2], [22], [1894], [944, 4], [1], [62], [184], [196], [513], [2], [678], [4733, 4], [4734], [6], [1895], [7], [184], [786], [580], [514], [], [4735, 4], [259], [515], [1], [4736, 4], [259], [

In [39]:
for i in range(0,len(x_test)):
  x_test[i] = process_text(x_test[i])

In [40]:
t  = Tokenizer()
t.fit_on_texts(x_test)
test_word_index = t.word_index
print("The word index", test_word_index)
test_sequences = t.texts_to_sequences(x_test)
print(test_sequences)

The word index {'في': 1, '0': 2, 'من': 3, '،': 4, 'و': 5, 'على': 6, 'إلى': 7, 'التي': 8, 'أو': 9, 'أن': 10, 'دمشق': 11, 'مع': 12, 'نظام': 13, 'مدريد': 14, 'لينكس': 15, 'الذي': 16, 'عام': 17, 'كان': 18, 'جنو': 19, 'اليورانيوم': 20, 'ريال': 21, 'حيث': 22, 'بعد': 23, 'راؤول': 24, 'ذلك': 25, 'عن': 26, 'عليه': 27, 'الطاقة': 28, 'unix': 29, 'ما': 30, 'هذه': 31, 'وفي': 32, 'النووية': 33, 'كل': 34, 'هذا': 35, 'سنة': 36, 'كانت': 37, 'تشغيل': 38, 'لا': 39, 'تم': 40, 'الذرة': 41, '2': 42, 'هي': 43, '1': 44, 'الدوري': 45, 'النظام': 46, 'العالم': 47, 'بطولة': 48, 'بين': 49, 'الفريق': 50, 'عدد': 51, 'بن': 52, 'مثل': 53, 'أوروبا': 54, 'السلام': 55, 'أهداف': 56, 'مباراة': 57, 'الإلكترونات': 58, 'هو': 59, 'وهو': 60, 'كما': 61, 'وقد': 62, 'النواة': 63, 'أكثر': 64, 'حتى': 65, '·': 66, 'لكن': 67, 'قبل': 68, 'مفاعل': 69, 'أبطال': 70, '3': 71, 'النووي': 72, 'دوري': 73, 'أي': 74, 'الأسباني': 75, 'هدف': 76, 'ذرات': 77, 'تلك': 78, 'لم': 79, 'مدينة': 80, 'العديد': 81, 'الحسين': 82, 'الماء': 83, 'نسخة': 84, 'بها

In [19]:
print(x_train)
print(x_test)

['كرستيانو' 'رونالدو' 'لاعب' ... 'بشكل' 'متزايد' '.']
['الذرة' 'هي' 'أصغر' ... 'طور' 'التصميم' '.']


In [41]:
lb = LabelBinarizer()
labels_train = lb.fit_transform(y_train)
lb = LabelBinarizer()
labels_test = lb.fit_transform(y_test)

In [42]:
t  = Tokenizer()
t.fit_on_texts(y_train)
y_train_word_index = t.word_index
print("The word index", y_train_word_index)

The word index {'o': 1, 'b': 2, 'i': 3, 'mis': 4, 'per': 5, 'loc': 6, 'org': 7, 'na': 8, 'io': 9, 'oo': 10}


In [43]:
y_unique= set(y_train)
print(y_unique)

{'', 'B-ORG', 'OO', 'I-LOC', 'na', 'B-MIS', 'O', 'IO', 'B-PER', 'I-ORG', 'I--ORG', 'B-LOC', 'I-MIS', 'I-PER'}


In [44]:
y_unique= set(y_test)
print(y_unique)

{'', 'B-ORG', 'I-LOC', 'na', 'B-MIS', 'O', 'B-PER', 'I-ORG', 'B-LOC', 'I-MIS', 'I-PER'}


In [147]:
LABELS= ['B-ORG' , 'OO' , 'O' , 'B-MIS' , 'B-PER' , 'I-ORG' , 'B-LOC' ,'I-MIS' ,'I-PER' , 'I-LOC' , 'na' ,'I--ORG' ,'IO' ]
unique_y_train = []
unique_y_test = []
for i in range(0,len(y_train)):
  for j in range(0,len(LABELS)) :
    if y_train[i] == LABELS[j] :
      unique_y_train.append(j)
    elif j >= len(LABELS):
      unique_y_train.append(11)

for i in range(0,len(y_test)):
  for j in range(0,len(LABELS)) :
    if y_test[i] == LABELS[j] :
      unique_y_test.append(j)
    if j >= len(LABELS):
      unique_y_test.append(11)

unique_y_train = np.array(unique_y_train)
unique_y_test = np.array(unique_y_test)

In [150]:
print(len(y_train))
print(len(unique_y_train))

48287
46271


In [53]:
print(y_test)
print(unique_y_test)

['O' 'O' 'O' ... 'O' 'O' 'O']
[2 2 2 ... 2 2 2]


In [156]:
flat_list = []
for sublist in sequences:
    for item in sublist:
        flat_list.append(item)
print(flat_list)

[1135, 50, 339, 2751, 4716, 1136, 2, 459, 47, 42, 27, 15, 5, 512, 6, 311, 339, 2, 22, 944, 4, 4717, 370, 4718, 3, 311, 3, 4719, 675, 2, 1137, 676, 27, 15, 1, 460, 183, 459, 184, 196, 513, 1420, 1, 41, 786, 580, 312, 1, 258, 2752, 313, 15, 83, 10, 2, 1421, 787, 55, 4720, 1891, 4721, 15, 139, 2753, 4722, 3, 2754, 258, 581, 17, 238, 2755, 1, 4723, 1, 83, 10, 788, 4724, 285, 123, 1, 55, 1138, 409, 4725, 4726, 2, 19, 677, 1139, 12, 2756, 4, 55, 30, 945, 4727, 3, 788, 1140, 4728, 4729, 2, 1141, 1892, 1893, 50, 7, 459, 4730, 677, 39, 4731, 340, 4732, 339, 2, 22, 1894, 944, 4, 1, 62, 184, 196, 513, 2, 678, 4733, 4, 4734, 6, 1895, 7, 184, 786, 580, 514, 4735, 4, 259, 515, 1, 4736, 4, 259, 679, 72, 410, 4737, 4738, 2, 152, 680, 946, 1135, 50, 2, 1142, 3, 1422, 12, 947, 4, 2, 2757, 1892, 789, 681, 4739, 2758, 4740, 4741, 4742, 4743, 4744, 1896, 97, 4745, 4746, 460, 2, 196, 513, 4747, 285, 4748, 7, 677, 2759, 4749, 786, 580, 6, 1423, 3, 8, 184, 4750, 5, 2760, 4751, 4752, 4753, 56, 1138, 1143, 1, 7

In [151]:
sentences = []
labels = []
index =-1
i=0
j=4
for word in range(len(flat_list)):
  if(j>= len(flat_list)):
    break
  index = index + 1
  sent= flat_list[i:j:]
  label = unique_y_train[i:j:]
  sentences.append(sent)
  labels.append(label)
  i=i+4
  j=j+4
print(len(sentences))
print(len(labels)) 

11143
11143


In [152]:
train_sentences = np.asarray(sentences)
train_labels = np.array(labels)
print(train_sentences.shape)
print(train_labels.shape)

(11143, 4)
(11143, 4)


In [153]:
print(train_sentences[0])
print(train_labels[0])

[1135   50  339 2751]
[4 8 2 2]


In [154]:
print(train_sentences)
print(train_labels)

[[ 1135    50   339  2751]
 [ 4716  1136     2   459]
 [   47    42    27    15]
 ...
 [  205     2   763   841]
 [  573  4689    17  1418]
 [    1   156   230 13262]]
[[4 8 2 2]
 [2 2 2 2]
 [0 5 2 2]
 ...
 [2 2 2 2]
 [2 2 3 2]
 [3 2 2 3]]


In [155]:
max_length = 4
padded_docs = pad_sequences(train_sentences, maxlen=max_length, padding='post')
print('padded_docs:\n',padded_docs)

padded_docs:
 [[ 1135    50   339  2751]
 [ 4716  1136     2   459]
 [   47    42    27    15]
 ...
 [  205     2   763   841]
 [  573  4689    17  1418]
 [    1   156   230 13262]]


In [157]:
test_flat_list = []
for sublist in test_sequences:
    for item in sublist:
        test_flat_list.append(item)
print(test_flat_list)

[41, 43, 290, 411, 3, 255, 658, 16, 2290, 2291, 659, 412, 255, 660, 882, 518, 163, 7, 661, 1328, 1329, 4, 2292, 164, 1330, 2293, 256, 18, 2294, 207, 662, 2295, 30, 59, 290, 3, 41, 291, 41, 3, 2296, 3, 2297, 1331, 58, 2298, 223, 98, 519, 883, 663, 224, 1, 884, 291, 63, 885, 31, 3, 886, 519, 883, 4, 5, 664, 1332, 41, 43, 290, 411, 3, 255, 93, 10, 887, 140, 26, 665, 1333, 256, 666, 2299, 64, 1, 225, 2300, 2301, 888, 889, 520, 141, 1334, 49, 345, 5, 129, 890, 4, 39, 1334, 49, 891, 1, 163, 2302, 5, 891, 129, 1, 163, 346, 667, 4, 9, 163, 74, 345, 129, 41, 4, 521, 1335, 3, 2303, 51, 2304, 4, 2305, 4, 2306, 1336, 4, 1337, 2307, 49, 257, 258, 4, 5, 49, 2308, 258, 892, 292, 2309, 893, 4, 5, 65, 49, 668, 35, 255, 2310, 6, 2311, 669, 1338, 30, 1339, 39, 894, 670, 41, 5, 30, 2312, 1, 35, 47, 1340, 2313, 4, 894, 5, 30, 895, 1341, 226, 5, 2314, 7, 1342, 2315, 5, 3, 896, 897, 898, 899, 227, 1, 293, 2316, 900, 2317, 5, 2318, 522, 85, 4, 2319, 2320, 2321, 1343, 2322, 4, 2323, 2324, 2325, 901, 4, 5, 2326

In [158]:
sentences = []
labels = []
index =-1
i=0
j=4
for word in test_flat_list:
  if(j>= len(test_flat_list)):
    break
  index = index + 1
  sent= test_flat_list[i:j:]
  label = unique_y_test[i:j:]
  sentences.append(sent)
  labels.append(label)
  i=i+4
  j=j+4
print(len(sentences))
print(len(labels)) 

4854
4854


In [159]:
print(sentences[0])
print(labels[0])

[41, 43, 290, 411]
[2 2 2 2]


In [160]:
test_sentences = np.array(sentences)
test_labels = np.array(labels)
print(test_sentences.shape)
print(test_labels.shape)

(4854, 4)
(4854, 4)


In [161]:
print(test_sentences)
print(test_labels)

[[  41   43  290  411]
 [   3  255  658   16]
 [2290 2291  659  412]
 ...
 [6596  320 6597  737]
 [   6 6598 6599   13]
 [  38  109  657   92]]
[[2 2 2 2]
 [2 2 2 2]
 [2 2 2 2]
 ...
 [7 2 2 2]
 [2 3 2 2]
 [2 2 2 2]]


In [162]:
max_length = 4
padded_docs = pad_sequences(test_sentences, maxlen=max_length, padding='post')
print('padded_docs:\n',padded_docs)

padded_docs:
 [[  41   43  290  411]
 [   3  255  658   16]
 [2290 2291  659  412]
 ...
 [6596  320 6597  737]
 [   6 6598 6599   13]
 [  38  109  657   92]]


In [163]:
model =  gensim.models.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/arabic_model.bin" , binary=True) 

In [164]:
vec=model['تكنولوجيا']
print(vec)

[ 0.323787  0.114331 -0.011856 -0.004447  0.047091  0.168247  0.074298
 -0.146852 -0.107001  0.348146  0.46207   0.741105  0.085863 -0.339043
 -0.480859  0.18958   0.301814 -0.210679  0.105646 -0.407463  0.42596
  0.050933  0.196187 -0.461667 -0.004837 -0.435574  0.0871   -0.059914
  0.839115  0.49071  -0.175257 -0.381858  0.187005  0.200588 -0.113988
 -0.248779 -0.060016 -0.643697 -0.159432 -0.404903  0.07752   0.267406
  0.210404  0.04395   0.676874  0.482013 -0.176632 -0.865179  0.086041
  0.091675 -0.18248  -0.208987 -0.255883 -0.210687 -0.009455 -0.191239
  0.293848 -0.166301  0.422393 -0.601161  0.301539 -0.234331  0.360695
  0.65819   0.252626  0.214151  0.188705 -0.043901 -0.108607 -0.072677
  0.148031  0.018153  0.6024   -0.287075 -0.311956  0.185124 -0.097128
  0.336573 -0.081013  0.208626  0.016204  0.172538  0.007021  0.15294
 -0.040931  0.094115 -0.414962  0.320592  0.302439 -0.108721  0.215081
  0.293792  0.853468  0.684285  0.372276 -0.000951 -0.478546  0.036926
  0.4590

In [166]:
w2v_embeddings_index={}
for word in model.wv.vocab:
    w2v_embeddings_index[word] = model[word]
print("Loaded %s word vectors."% len(w2v_embeddings_index))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Loaded 1071056 word vectors.


In [168]:
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = w2v_embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Embedding Matrix shape: ', embedding_matrix.shape)

Embedding Matrix shape:  (13263, 100)


In [169]:
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=4,
                            trainable=False)

In [230]:
input= Input(shape=(4,), dtype='int32')
input_embedding = embedding_layer(input)
lstm= LSTM(units=20, activation='softmax')(input_embedding)
preds = Dense(4, activation='softmax' , name = 'activation')(lstm)
model = Model(inputs=input, outputs=preds)

model.compile(loss="categorical_crossentropy", optimizer= Adam(lr=0.0001 ), metrics=['accuracy'])
print(model.summary())

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 4)]               0         
                                                                 
 embedding (Embedding)       (None, 4, 100)            1326300   
                                                                 
 lstm_22 (LSTM)              (None, 20)                9680      
                                                                 
 activation (Dense)          (None, 4)                 84        
                                                                 
Total params: 1,336,064
Trainable params: 9,764
Non-trainable params: 1,326,300
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [231]:
history= model.fit(train_sentences, train_labels, epochs=20, verbose=1 , batch_size=50, validation_data=(test_sentences, test_labels))

Epoch 1/20
223/223 [==============================] - 3s 7ms/step - loss: 13.2223 - accuracy: 0.0792 - val_loss: 13.6182 - val_accuracy: 0.0979
Epoch 2/20
223/223 [==============================] - 1s 6ms/step - loss: 13.2225 - accuracy: 0.0757 - val_loss: 13.6183 - val_accuracy: 0.0966
Epoch 3/20
223/223 [==============================] - 1s 6ms/step - loss: 13.2228 - accuracy: 0.0757 - val_loss: 13.6186 - val_accuracy: 0.0960
Epoch 4/20
223/223 [==============================] - 1s 6ms/step - loss: 13.2229 - accuracy: 0.1227 - val_loss: 13.6184 - val_accuracy: 0.3169
Epoch 5/20
223/223 [==============================] - 1s 6ms/step - loss: 13.2229 - accuracy: 0.6767 - val_loss: 13.6190 - val_accuracy: 0.7347
Epoch 6/20
223/223 [==============================] - 1s 5ms/step - loss: 13.2236 - accuracy: 0.7865 - val_loss: 13.6199 - val_accuracy: 0.7371
Epoch 7/20
223/223 [==============================] - 1s 5ms/step - loss: 13.2242 - accuracy: 0.7887 - val_loss: 13.6204 - val_accuracy:

In [308]:
loss, accuracy = model.evaluate(train_sentences, train_labels, verbose=1)
print("Accuracy: %f" % (accuracy*100))

349/349 [==============================] - 1s 2ms/step - loss: 13.2260 - accuracy: 0.7900
Accuracy: 79.000270


In [306]:
text=['الاقصي فلسطين افضل']
t  = Tokenizer()
t.fit_on_texts(text)
sequences_text = t.texts_to_sequences(text)
print(sequences_text)
padded_text = pad_sequences(sequences_text, maxlen=4, padding='post')
print('padded_text:\n',padded_text)

[[1, 2, 3]]
padded_text:
 [[1 2 3 0]]


In [307]:
result=model.predict(padded_text)
result_text = []
for sublist in result:
    for item in sublist:
        result_text.append(item)
print('Input %s \n Prediction: %s' %(text,result_text))

Input ['الاقصي فلسطين افضل'] 
 Prediction: [0.25477383, 0.24642503, 0.2490997, 0.2497014]


In [305]:
for i in range(0,len(result_text)):
  result_text[i] = np.around(result_text[i],0)
  result_text[i] = LABELS[result_text[i].astype(int)]
print(result_text)

['B-ORG', 'B-ORG', 'B-ORG', 'B-ORG']
